In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

import sys
sys.path.append("../")

from kramersmoyal.kmc import kmc_kernel_estimator
from kramersmoyal.kernels import epanechnikov, gaussian

## KMC

### 1d

In [ ]:
timeseries = np.random.normal(loc=0.0, scale=1,size=[10_000_00,1])
# timeseries = np.random.rand(10_000_000, 1)

bins = np.array([200])

powers = np.array([[0]])
# powers = np.array([[0,1,0,1,2,0,2,4,0,4,6,0,6,8,0]])

kmc, edges = kmc_kernel_estimator(
    timeseries, kernel=epanechnikov, bw=0.01, bins=bins, powers=powers)

In [ ]:
%matplotlib inline

X, = np.meshgrid(*edges)

plt.plot(X, kmc[...,0])

### 2d

In [ ]:
timeseries = np.random.normal(loc=0, scale=1,size=[10_000_000,2])
# timeseries = np.random.rand(1_000_000, 2)

bins = np.array([200,200])

powers = np.array([[0],[0]])
# powers = np.array([[0,1,0,1,2,0,2,4,0,4,6,0,6,8,0], [0,0,1,1,0,2,2,0,4,4,0,6,6,0,8]])

kmc, edges = kmc_kernel_estimator(
    timeseries, kernel=epanechnikov, bw=1, bins=bins, powers=powers)

In [ ]:
%matplotlib notebook

fig = plt.figure()
ax = Axes3D(fig)

X, Y = np.meshgrid(*edges)

ax.plot_surface(X, Y, kmc[...,0].T,
                alpha=0.85, rstride=10, cstride=10, linewidth=2, 
                antialiased=True, color='white', zorder = .1)